# Data used to perform the analysis
The precidency of Donald J. Trump began at noon EST (17:00 UTC) on January 20, 2017 when he was inaugurated as the 45th president of the United States, and will com to an end the on January 20, 2021 as he ultimately lost the 2020 presidential election to Joe Biden. It is not far fetched to say it has been bizare precidency compared to the most recent precidencies. 

It feels like America has been splitted in two the suporters of Trump and those agianst him - Repulicans agianst Democrats. In this project we wanted to explore if our hypothesis of polarization can be seen or rejected by analyzing the Congress of United States behavoir on the social media Twitter including the infamous Twitter account manged by Donald J. Trump. The idea was to analyze the congress tweets from the time of the 45th precidency to explore potential polarization.

With access to the Twitter API it is only possible to exctract the most reason 3200 tweets from a given account (3200 tweet do not go far back for many american polticians). However, Twitter's Terms of Service do allow for datasets of tweets ID's to be distributed to third parties (not the full JSON). Luckily we found two sources that keep tweet id open very related to our project and one sources that stored the full length tweets of Donald Trump namely:

* **115th U.S. Congress Tweet Ids:**
    A open dataset with 2,041,399 tweets from the Twitter accounts of members of the 115th U.S. Congress collected in the period of January 27, 2017 and January 2, 2019. The dataset consists two files of interest namely
    * `senators-1.txt` that contains tweet ids for Seneators
    * `representatives-1.txt` that contains tweet ids for Representatives
    *Littman, Justin, 2017, "115th U.S. Congress Tweet Ids", https://doi.org/10.7910/DVN/UIVHQR, Harvard Dataverse, V5.*

* **116th U.S. Congress Tweet Ids**
    A open dataset with 2,817,747 tweets from the Twitter accounts of members of the 116th U.S. Congress collected in the period of January 27, 2019 and May 7, 2020.  The dataset consists two files of interest namely
    * `Senators: congress116-senate-ids.txt` that contains tweet ids for Seneators
    * `Representatives: congress116-house-ids.txt` that contains tweet ids for Representatives  * Wrubel, Laura; Kerchner, Daniel, 2020, "116th U.S. Congress Tweet Ids", https://doi.org/10.7910/DVN/MBOJNS, Harvard Dataverse*

* **Trump Twitter Archive**
    A site dedicated to scrape every single tweet from Donald J. Trump. Here we downloaded all tweets in the periods of January 27, 2017 and January 2, 2019 and January 27, 2019 and May 7, 2020. https://www.thetrumparchive.com/

Examing the Harvard Dataverse we discovered that polticians in congress can have number of profiles for instance a private profile, a profile associated with work in congress and campaign profile. Unfortunally also a number of random profiles like support profile for certain candidate. Moreover there is tweet dateing as far back as 2008. Furthermore it was not listed what party the account where associated with. However, we found a list of congress memebers with their party association, whether they act as Senator or Represenative and, most important for the project, their twitter profile (if they have an account the vast majority has). That meant rather than dealing with multiple accounts for the same congress member or including random account we would only consider one account per congress member. Two different sources for the 115 and 116 congress have been utilized for this namely

* 116. Congress twitter info: (website) https://triagecancer.org/congressional-social-media 
* 115. Congress twitter info : (PDF) https://www.sciencecoalition.org/wp-content/uploads/2018/09/115th-Congress-Twitter-Handles.pdf  

The politicans listed in these to documents as well as Donald Trump are the twitter profiles that ones that will considered. The information needs to scraped as the format is HTML and PDF respectively.

## Handeling the Data

This notebook will explain how the above described data was exstracted and preprocessed. The notebook consist of four parts and one smaller part devoted to extrating Trump tweets id (part 0). So in total:
* **Part 0: Exstract Trump Tweet IDs**
    Here the tweet id are exstacted from the tweets made publicly by https://www.thetrumparchive.com/

* **Part 1: Hydrate Tweets**
    In this part all the tweet ids from Harvard data archive as well as Trumps tweet id, are hydrated. I.e. the ids are turned back into tweets with metadata. 

* **Part 2: Getting Congress Twitter Account**
    Here a pandas data frame is constructed from information scraped from the PDF describing the 115 Congress memebers twitter accounts, and the HTML site describing the 116 Congress Twitter info.

* **Part 3: Clean-up of Harward Data**
    The Harvard data archive needs to be cleaned prior to analsis as it contains
    * Data prior to January 27, 2017
    * Duplicates
    * Remove random/duplicate twitter accounts
    
* **Part 4: Preprocess the twitter data**
    Many tweets contains links, emojies etc that makes it difficult to perform natural language processing on. In this part the tweets are preprocessed such that they can be used for our analysis.
    
    
Below can all the library dependicies be read. Note the import of functions from our own module `src`. The source code for these functions can be read on our github repository https://github.com/MikkelGroenning/02805_social_graphs.

In [2]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import requests
import urllib.request
import camelot
import tweepy
import tqdm
from src.data.trump_tweet_ids import get_trump_tweet_ids
from src.data.hydrate import hydrate_tweets
from src.tools.twitter_api_credentials import api_key, api_secret_key, access_token, access_token_secret

The file twitter_api_credentials.py can't be found in out Github repository as it contains classified information about our creditiels to the Twitter API. To recreate the dataset one needs to access the Twitter API through developer account is needed. Such an account can be requested at https://developer.twitter.com/en/apply-for-access typically one is granted access instant.

In [2]:
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
try:
    redirect_url = auth.get_authorization_url()
except tweepy.TweepError:
    print('Error! Failed to get request token.')

## Part 0: Exstract Trump Tweet IDs
As the site Trump Twitter Achive (https://www.thetrumparchive.com/) store Donald Trump's tweets in a different format than how it typically exstracted from Twitter-API we exstracted the tweet id from this sources and stored them in the file `trump_id.txt`.

In [4]:
df_trump_tweets1 = pd.read_csv('../Data/raw/tweets/trump_tweets_1st.csv')  
df_trump_tweets2 = pd.read_csv('../Data/raw/tweets/trump_tweets_1st.csv')
df_trump = pd.concat([df_trump_tweets1, df_trump_tweets2])

In [6]:
filepath = "../Data/raw/tweets/trump_id.txt"
get_trump_tweet_ids(df_trump, filepath)

11326 tweet ids saved


## Part 1: Hydrate Tweets
The process of turning tweet ID's into actual tweets with metadata is called *hydration* and requires Twitter delopper account. In the below cell we load all twitter ids obtained from Harward Data Archive and Trump Twitter Archive.

In [5]:
representatives115 = np.loadtxt(
    "../Data/Raw/Tweets/representatives115.txt", dtype=int
)
representatives116 = np.loadtxt(
    "../Data/Raw/Tweets/representatives116.txt", dtype=int
)
senators115 = np.loadtxt(
    "../Data/Raw/Tweets/senators115.txt", dtype=int
)
senators116 = np.loadtxt(
    "../Data/Raw/Tweets/senators116.txt", dtype=int
)
trump = np.loadtxt(
    "../Data/Raw/Tweets/trump_id.txt", dtype=int
)
congress = np.concatenate([representatives115, representatives116, senators115, senators116, trump])
print(len(congress))

4870472


The concatted into array of tweet id consist of 4.8 millions ID. All these tweet are now hydrated with the function `hydrate_tweets` located in src/data folder in our reposortiry. Note running the cell below take $24 \pm 6$ hours as the twitter API set limits to how much can be exstracted. More info about rate limits can be found at https://developer.twitter.com/en/docs/twitter-api/v1/rate-limits

In [ ]:
filepath = "../Data/interim/congress.pkl"
api = tweepy.API(auth, wait_on_rate_limit=True)

hydrate_tweets(
    tweet_ids=congress,
    filepath=filepath,
    api = api
)

## Part 2: Getting Congress Twitter Account
In this part a pandas data frame will generated with each members congress member's name State, Type (Reprensative, sSnator, POTUS), Name, Party. This part cosist of three subparts:
* **Part 2.1: 116<sup>th</sup>** Here the desired data frame for 116 congress will be scraped
* **Part 2.2: 115<sup>th</sup>** Here the desired data frame for 115 congress will be scraped
* **Part 2.3: Merge data** Here the different congress data frame will be merged.

### Part 2.1: 116<sup>th</sup> congress

First the twitter handles for the 116<sup>th</sup> congress will be extracted using [this](https://triagecancer.org/congressional-social-media) source. The choice of source comes from the fact that the Twitter handle as well as the party is desired.

`BeautifulSoup` is used to extract the HTML table from the webpage (that has been downloaded to allow for offline work).

In [ ]:
# Open data
with open('../Data/Raw/116_congress_twitter.html') as fp:
    soup = BeautifulSoup(fp, 'html.parser')

# Find table
table = soup.find('table', attrs={'id':"footable_16836"})

# Extract data row wise from table
l = []
for tr in table.findAll('tr'):
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)

# Make the data into a Pandas data frame and drop irrelevant columns
Data116 = pd.DataFrame(l[1:], columns = [header.getText() for header in table.findAll('th')]).drop(columns = ['Name Links', 'Twitter Links', 'Instagram', 'Facebook Page', 'Facebook'])

# Ensure that the type of politician is alligned
rename_chamber = {'U.S. Representative': 'Representative', 'U.S. Senator': 'Senator'}
Data116 = Data116.replace(rename_chamber).rename(columns = {'Chamber of Congress': 'Type'})

In this data set the state is given as well as congressional district. This is fixed using regex strings as shown below. Moreover the "@" are removed from the Twitter handles as the Twitter API does not need it. The vancant positions in Congress are also disregarded.

In [17]:
# All states abbreviations
us_state_abbrev = {
    r'Alabama.*': 'AL',
    r'Alaska.*': 'AK',
    r'American Samoa.*': 'AS',
    r'Arizona.*': 'AZ',
    r'Arkansas.*': 'AR',
    r'California.*': 'CA',
    r'Colorado.*': 'CO',
    r'Connecticut.*': 'CT',
    r'Delaware.*': 'DE',
    r'District of Columbia.*': 'DC',
    r'Florida.*': 'FL',
    r'Georgia.*': 'GA',
    r'Guam.*': 'GU',
    r'Hawaii.*': 'HI',
    r'Idaho.*': 'ID',
    r'Illinois.*': 'IL',
    r'Indiana.*': 'IN',
    r'Iowa.*': 'IA',
    r'Kansas.*': 'KS',
    r'Kentucky.*': 'KY',
    r'Louisiana.*': 'LA',
    r'Maine.*': 'ME',
    r'Maryland.*': 'MD',
    r'Massachusetts.*': 'MA',
    r'Michigan.*': 'MI',
    r'Minnesota.*': 'MN',
    r'Mississippi.*': 'MS',
    r'Missouri.*': 'MO',
    r'Montana.*': 'MT',
    r'Nebraska.*': 'NE',
    r'Nevada.*': 'NV',
    r'New Hampshire.*': 'NH',
    r'New Jersey.*': 'NJ',
    r'New Mexico.*': 'NM',
    r'New York.*': 'NY',
    r'North Carolina.*': 'NC',
    r'North Dakota.*': 'ND',
    r'Northern Mariana Islands.*':'MP',
    r'Ohio.*': 'OH',
    r'Oklahoma.*': 'OK',
    r'Oregon.*': 'OR',
    r'Pennsylvania.*': 'PA',
    r'Puerto Rico.*': 'PR',
    r'Rhode Island.*': 'RI',
    r'South Carolina.*': 'SC',
    r'South Dakota.*': 'SD',
    r'Tennessee.*': 'TN',
    r'Texas.*': 'TX',
    r'Utah.*': 'UT',
    r'Vermont.*': 'VT',
    r'Virgin Islands.*': 'VI',
    r'Virginia.*': 'VA',
    r'Washington.*': 'WA',
    r'West V.*': 'WV', # Written in different ways
    r'Wisconsin.*': 'WI',
    r'Wyoming.*': 'WY'
}

# Convert states to two letter abbreviations
Data116['State'] = Data116['State'].replace(regex = us_state_abbrev)

# Remove @
Data116 = Data116.replace(regex = {r'^@': ''})

# Remove vacant positions
Data116 = Data116[Data116.Name != "Vacant"]

# Look at the data
Data116

,State,Type,Name,Party,Twitter
0,AL,Senator,Richard Shelby,R,SenShelby
1,AL,Senator,Doug Jones,D,DougJones
2,AL,Representative,"Byrne, Bradley",R,RepByrne
3,AL,Representative,"Roby, Martha",R,RepMarthaRoby
4,AL,Representative,"Rogers, Mike",R,RepMikeRogersAL
...,...,...,...,...,...
536,WI,Representative,"Tiffany, Thomas",R,TomTiffanyWI
537,WI,Representative,"Gallagher, Mike",R,MikeforWI
538,WY,Senator,"Enzi, Mike",R,SenatorEnzi
539,WY,Senator,"Barrasso, John",R,SenJohnBarrasso


It is also seen that there are an inconsistency in the ways the names are written. This is changed so all names are written with the first name first:

In [18]:
Data116['Name'] = [name[1][1:]+ " " +name[0] if len(name) == 2 else name[0] for name in [name.replace(u'\xa0', u'').split(',') for name in Data116.Name]]

### Part 2.2: 115<sup>th</sup> congress

Now we move onto the 115th congress. This is data stored in a pdf.table, so for this the `camelot` library is used. 

In [19]:
# Get data
file115 = '../Data/Raw/115_congress_twitter.pdf'

# Read table across all pages
tables = camelot.read_pdf(file115, pages = 'all')

# Convert data to pandas data frame
Data115 = pd.DataFrame(np.concatenate([d.df.drop(0).values for d in tables]), columns=tables[0].df.iloc[0]).drop(columns = "District")

# Align chamber name with the 116 data
rename_chamber = {'Rep.': 'Representative', 'Sen.': 'Senator'}
Data115 = Data115.replace(rename_chamber)

# Align name with the 116 data and store it in one column
Data115["Name"] = Data115["First Name"] + " " + Data115["Last Name"]
Data115 = Data115.drop(columns = ["First Name", "Last Name"])

# Align columns name with the 116 data
Data115 = Data115.rename(columns = {'Title': 'Type', "Twitter Handle": "Twitter"})

### Part 2.3: Merge data

Now the two datasets are merged. Here we need to take duplicate acounts into account which accounts for reelections.

In [20]:
# Merge data set
Data_Full = Data115.append(Data116, ignore_index = True)

# Get shape
Data_Full.shape

(1072, 5)

In [21]:
api = tweepy.API(auth, wait_on_rate_limit=True)
to_remove = []
twitter_display_name = []
for index, handle in tqdm.tqdm(enumerate(Data_Full.Twitter)):
    try:
        u=api.get_user(handle)
    except Exception:
        to_remove.append(index)

1072it [16:04,  1.11it/s]


In [22]:
Data_Full = Data_Full.drop(index=to_remove)


In [24]:
# Extra duplicate from AS
Data_Full = Data_Full[Data_Full.Twitter != 'RepTomPrice']

# Drop closed users
Data_Full = Data_Full[~Data_Full.Name.isin(['Aumua Radewages', 'Madeleine Bordallo', 'Elizabeth Esty'])]

# Fix Eric
Data_Full.loc[Data_Full[Data_Full.Name == "Erik Paulsen"].index,"Twitter"] = "ErikPaulsen"

# Fix Bobby
Data_Full.loc[Data_Full[Data_Full.Name == "Bobby Scott"].index,"Twitter"] = "BobbyScott"

# Fix Dave
Data_Full.loc[Data_Full[Data_Full.Name == 'Dave Reichert'].index,"Twitter"] = "TeamReichert"

# Fix Lindsey
Data_Full.loc[Data_Full[Data_Full.Name == 'Lindsey Graham'].index,"Twitter"] = "LindseyGrahamSC"

# Darin's name
Data_Full.loc[Data_Full[Data_Full.Name == "arin LaHood"].index,"Name"] = "Darin LaHood"

In [26]:
Data_Full = Data_Full.drop_duplicates(subset = ["Twitter"], keep = 'last')

In [28]:
Data_Full = Data_Full.drop_duplicates(subset = ["Name"], keep = 'last')

Add the President

In [30]:
Data_Full = Data_Full.append({'State': None, 'Party': 'R', 'Type': 'POTUS', 'Twitter': 'realDonaldTrump', 'Name': 'Donald J. Trump', 'twitter_display_name': 'Donald J. Trump'}, ignore_index=True)

In [31]:
twitter_display_name = [api.get_user(handle).name for handle in Data_Full.Twitter]
Data_Full['twitter_display_name'] = twitter_display_name

In [32]:
Data_Full.to_csv('../Data/Processed/Twitter_Handles.csv')

## Part 3: Clean-up of Harward Data

In [3]:
congress = pd.read_pickle('../Data/Interim/congress.pkl')
twitter_handles = pd.read_table('../Data/Processed/Twitter_Handles_updated.csv', sep = ',')

s1 = set(twitter_handles['twitter_display_name'])
s2 = set(congress.user_name.unique())

In [5]:
non_overlapping_twitter_profiles = s1 ^ s2
non_overlapping_twitter_profiles

{'#TonyCardenas🇺🇸',
 '(((Jerry Nadler)))',
 '@JasonSmithMO',
 'Abigail Spanberger',
 'Adam Smith',
 'Alex Mooney',
 'Alexandria Ocasio-Cortez',
 'Andy Barr',
 'Andy Harris',
 'Andy Kim',
 'Andy Levin',
 'Ann Kirkpatrick',
 'Annie Kuster',
 'Anthony Brindisi',
 'Anthony G. Brown',
 'Anthony Gonzalez',
 'Antonio Delgado',
 'Archive: Senator McCaskill Office',
 'Archive: Tom Graves',
 'Ayanna Pressley',
 'Barbara Lee',
 'Barry Loudermilk',
 'Ben Cardin',
 'Ben Cline',
 'Ben McAdams',
 'Ben Sasse',
 'Bill Flores',
 'Bill Keating',
 'Billy Long',
 'Bishop for Congress',
 'Blake Farenthold',
 'Bob Casey Jr.',
 'Bob Latta',
 'Bobby Scott',
 'Brad Schneider',
 'Brad Sherman',
 'Bradley Byrne',
 'Brett Guthrie',
 'Brian Fitzpatrick',
 'Brian Schatz',
 'Bruce Westerman',
 'Capito for WV',
 'Carlos Curbelo',
 'Carolyn B. Maloney',
 'Cathy McMorris Rodgers',
 'Cedric Richmond',
 'Charlie Dent',
 'Cheri Bustos',
 'Chip Roy',
 'Chris Pappas',
 'Chris Stewart',
 'Chris Van Hollen',
 'Cindy Axne',
 'C

In [ ]:
# Make sure tweets only comes from people that twitter handles exist for. 
congress = congress[congress.user_name.isin(s1)]

In [ ]:
# Keep only the periods from Harward:
mask = (
    #January 27, 2017 and January 2, 2019 
    (congress.created_at > '2017-1-27 00:00:00') & (congress.created_at < '2019-1-2 00:00:00')
    | 
    #January 27, 2019 and May 7, 2020 
    (congress.created_at > '2019-1-27 00:00:00') & (congress.created_at < '2020-5-7 00:00:00')
)
congress = congress[mask]

In [ ]:
congress = congress.drop_duplicates(keep='first')
congress = congress.sort_values(by='created_at')
congress = congress.reset_index(drop=True)
congress.to_pickle("Data/interim/congress_cleaned.pkl")

In [ ]:
# Extract the tweets ids and convert them to integers
ids = list(congress.id.astype(int).values)

filepath = "../Data/raw/tweets/Cleaned_tweet_id.txt"
with open(filepath, 'w') as output:
    for row in ids:
        output.write(str(row) + '\n')

    print(f'{len(ids)} tweet ids saved.')

### Part 2.1 Shortcut to exstract the data

The dataframe with tweets of from congress after cleanup contain 60 % rows. 

Note running the cell below take $10 \pm 2.5$ hours as the twitter API set limits to how much can be exstracted.

In [ ]:
congress_tweet_id = np.loadtxt("../Data/Raw/Tweets/Cleaned_tweet_id.txt", dtype=int)
filepath = "../Data/interim/congress_cleaned.pkl"

hydrate_tweets(
    tweet_ids=congress_tweet_id,
    filepath=filepath,
    api = api
)

## Part 3: Preprocess the twitter data

In [ ]:
congress = pd.read_pickle('../Data/Interim/congress_cleaned.pkl')

In [ ]:
special_characters = ",._´&’%':€$£!?#"
character_set = {
    "characters": "abcdefghijklmnopqrstuvwxyz0123456789" + special_characters,
    "space": " ",
}
alphabet = "".join(character_set.values())

In [ ]:
regex_links = re.compile("http\S+")
regex_whitespace = re.compile("[\s|-]+")
regex_unknown = re.compile(f"[^{alphabet}]+")

In [ ]:
regex_html_tags = {
    "&amp": "and",
    "&lt": "<",
    "&gt": ">",
    "&quot": '"',
    "&apos": "'",
}

In [ ]:
## Replace unicode charetars
for pattern_string, char in regex_html_tags.items():
    congress_tweets["text"] = congress_tweets["text"].str.replace(pattern_string, char)

In [ ]:
congress_tweets["text"] = (congress_tweets["text"]
    .str.lower()
    .str.replace(regex_links, "")
    .str.replace(regex_whitespace, character_set["space"])
    .str.replace(regex_unknown, '')
    .str.strip()
)

In [ ]:
congress_tweets.to_pickle('../Data/Processed/congress_cleaned_processed.pkl')